In [17]:
import json
import pandas as pd

In [3]:
trust_score_weight = [0.3, 0.7]

# Compute Foursquare trust score for each tuple

In [4]:
def open_fs_json(fs_file_name):
    """Read foursquare json files
    input: foursquare file name
    output: json_formatted restaurant tuples
    """
    with open(fs_file_name) as fs_json:
        fs_restaurants = json.load(fs_json)
    return fs_restaurants

In [5]:
# load foursquare restaurant tuples for each data collection day: 0324, 0331, 0406
fs_restaurants_0324 = open_fs_json('Semiar_FS_0324.json')
fs_restaurants_0331 = open_fs_json('Semiar_FS_0331.json')
fs_restaurants_0406 = open_fs_json('Semiar_FS_0406.json')

In [6]:
def json_to_checkin(fs_restaurants):
    """extract checkins for each restaurant tuple
    input: json_formatted restaurant tuples
    output: list of restaurant-wise checkins
    """
    restaurant_checkins_list = []
    for restaurant in fs_restaurants:
        checkin = restaurant['response']['venue']['stats']['checkinsCount']
        restaurant_checkins_list.append(checkin)
    return restaurant_checkins_list

In [7]:
# generate foursquare restaurant checkin lists for each data collection day: 0324, 0331, 0406
restaurant_checkins_list_0324 = json_to_checkin(fs_restaurants_0324)
restaurant_checkins_list_0331 = json_to_checkin(fs_restaurants_0331)
restaurant_checkins_list_0406 = json_to_checkin(fs_restaurants_0406)

In [8]:
def checkin_to_diff(checkins_lists, durations):
    """generate checkin differences for each duration of data collection
    input: all checkins lists, length(number of days) of each duration
    ouput: lists of checkins differences
    
    """
    checkins_diffs_list = []
    for lists_idx in range(len(checkins_lists)-1):
        # a pair of checkins is zipped together for computational convenience
        list_pair_zipped = list(zip(checkins_lists[lists_idx],checkins_lists[lists_idx+1]))
#         checkins_diff = []
        for pair_idx in range(len(list_pair_zipped)):# the element update should be assisted by index
        # daily difference
            list_pair_zipped[pair_idx] \
            = (list_pair_zipped[pair_idx][1]-list_pair_zipped[pair_idx][0])/durations[lists_idx]
        checkins_diffs_list.append(list_pair_zipped)
    return checkins_diffs_list

In [9]:
checkins_lists = [restaurant_checkins_list_0324, restaurant_checkins_list_0331, restaurant_checkins_list_0406]
durations = [7,6]

In [10]:
# lists of checkins differences
checkins_diffs_list = checkin_to_diff(checkins_lists, durations)

In [11]:
# zipped difference together
checkins_diffs_zipped = list(zip(*checkins_diffs_list))

In [12]:
# Foursquare
# sum up the daily difference to obtain the trust value for each tuple
trust_scores_list_FS = []
for diffs in checkins_diffs_zipped:
    trust_score_FS = trust_score_weight[0]*abs(diffs[0]) + trust_score_weight[1]*abs(diffs[1])
    trust_scores_list_FS.append(trust_score_FS)

In [14]:
# trust score for the first 5 restaurants
trust_scores_list_FS[0:5]

[0.04285714285714285, 0.0, 0.2333333333333333, 0.11666666666666665, 0.0]

In [18]:
# dataframe to save trust scores
trust_scores_df = pd.DataFrame({'trust_score_FS':trust_scores_list_FS})

In [20]:
trust_scores_df.to_pickle('trust_scores_df')

# Compute Yelp trust score for each tuple

In [ ]:
    with open(fs_file_name) as fs_json:
        fs_restaurants = json.load(fs_json)